Yapacağımız Credit Card Clustering uygulaması için modellerimizi kaydedeceğiz

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [1]:
import pandas as pd

In [2]:
# Veriyi Yüklüyoruz

In [3]:
df = pd.read_csv("CC GENERAL.csv")

In [5]:
df.head()

,CUST_ID,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
0,C10001,40.900749,0.818182,95.40,0.00,95.4,0.000000,0.166667,0.000000,0.083333,0.000000,0,2,1000.0,201.802084,139.509787,0.000000,12
1,C10002,3202.467416,0.909091,0.00,0.00,0.0,6442.945483,0.000000,0.000000,0.000000,0.250000,4,0,7000.0,4103.032597,1072.340217,0.222222,12
2,C10003,2495.148862,1.000000,773.17,773.17,0.0,0.000000,1.000000,1.000000,0.000000,0.000000,0,12,7500.0,622.066742,627.284787,0.000000,12
3,C10004,1666.670542,0.636364,1499.00,1499.00,0.0,205.788017,0.083333,0.083333,0.000000,0.083333,1,1,7500.0,0.000000,NaN,0.000000,12
4,C10005,817.714335,1.000000,16.00,16.00,0.0,0.000000,0.083333,0.083333,0.000000,0.000000,0,1,1200.0,678.334763,244.791237,0.000000,12


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8950 entries, 0 to 8949
Data columns (total 18 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   CUST_ID                           8950 non-null   object 
 1   BALANCE                           8950 non-null   float64
 2   BALANCE_FREQUENCY                 8950 non-null   float64
 3   PURCHASES                         8950 non-null   float64
 4   ONEOFF_PURCHASES                  8950 non-null   float64
 5   INSTALLMENTS_PURCHASES            8950 non-null   float64
 6   CASH_ADVANCE                      8950 non-null   float64
 7   PURCHASES_FREQUENCY               8950 non-null   float64
 8   ONEOFF_PURCHASES_FREQUENCY        8950 non-null   float64
 9   PURCHASES_INSTALLMENTS_FREQUENCY  8950 non-null   float64
 10  CASH_ADVANCE_FREQUENCY            8950 non-null   float64
 11  CASH_ADVANCE_TRX                  8950 non-null   int64  
 12  PURCHA

In [7]:
df.isnull().sum()

CUST_ID                               0
BALANCE                               0
BALANCE_FREQUENCY                     0
PURCHASES                             0
ONEOFF_PURCHASES                      0
INSTALLMENTS_PURCHASES                0
CASH_ADVANCE                          0
PURCHASES_FREQUENCY                   0
ONEOFF_PURCHASES_FREQUENCY            0
PURCHASES_INSTALLMENTS_FREQUENCY      0
CASH_ADVANCE_FREQUENCY                0
CASH_ADVANCE_TRX                      0
PURCHASES_TRX                         0
CREDIT_LIMIT                          1
PAYMENTS                              0
MINIMUM_PAYMENTS                    313
PRC_FULL_PAYMENT                      0
TENURE                                0
dtype: int64

In [4]:
# Veri Ön İşleme

In [8]:
if 'CUST_ID' in df.columns:
    df.drop('CUST_ID', axis=1, inplace=True) # CUST_ID gereksiz diye attık

In [9]:
# Eksik verileri doldurma (Medyan ile)

In [ ]:
df['MINIMUM_PAYMENTS'].fillna(df['MINIMUM_PAYMENTS'].median(), inplace=True)

In [ ]:
df['CREDIT_LIMIT'].fillna(df['CREDIT_LIMIT'].median(), inplace=True)

In [ ]:
# Ölçeklendirme (StandardScaler)

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()

In [ ]:
X_scaled = scaler.fit_transform(df)

In [ ]:
# En İyi K Değerini Bulma (Silhouette Score ile)

In [ ]:
best_k = 2

In [ ]:
best_score = -1

In [ ]:
best_model = None

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
from sklearn.metrics import silhouette_score

In [ ]:
for k in range(2, 7): # 2 ile 6 küme arasını dene
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = kmeans.fit_predict(X_scaled)
    score = silhouette_score(X_scaled, labels)
    
    print(f"k={k} için Silhouette Score: {score:.4f}")
    
    if score > best_score:
        best_score = score
        best_k = k
        best_model = kmeans

In [ ]:
print("-" * 30)
print(f"🏆 EN İYİ MODEL: k={best_k}")
print(f"📊 BAŞARI SKORU (Silhouette): {best_score:.4f}")

In [ ]:
# En İyi Modeli save

In [ ]:
df['Cluster'] = best_model.labels_

In [ ]:
# PCA ile 2 boyuta indirgiyoruz (Görselleştirme için)

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(n_components=2)

In [ ]:
principal_components = pca.fit_transform(X_scaled)

In [ ]:
df['PCA1'] = principal_components[:, 0]

In [ ]:
df['PCA2'] = principal_components[:, 1]

In [ ]:
# Dosyaları Kaydet

In [ ]:
df.to_csv("credit_card_clustered.csv", index=False)

In [ ]:
import joblib

In [ ]:
joblib.dump(best_model, "cc_kmeans_model.pkl")

In [ ]:
joblib.dump(scaler, "cc_scaler.pkl")

In [ ]:
# uygulamada gösterebilmek için skoru txt yapıyoruz

In [ ]:
with open("model_score.txt", "w") as f:
    f.write(f"{best_score:.4f}")